<a href="https://colab.research.google.com/github/hamsterjoa/Kbank_playstore_review_analysis/blob/master/Kbank_App_review_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length{length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving review_dataset.csv to review_dataset.csv
User uploaded file "review_dataset.csv" with length553990 bytes


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install customized_konlpy

In [ ]:
import io
import pandas as pd
import numpy as np

import re
import nltk
import pickle
from konlpy.tag import Okt; t=Okt()
from tqdm import tqdm
from pykospacing import Spacing
from collections import Counter
import urllib.request
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv(io.StringIO(uploaded['review_dataset.csv'].decode('utf-8')))
df.drop(['Unnamed: 0','id','dateYear','dateMonth','dateDay'], axis = 1, inplace=True)
# strict한 기준으로 판단하기 위해 2점까지는 label 0, 3점부터는 label 1
df['label'] = np.select([df.rating>2],[1],default=0)
# 데이터 확인
df

,date,rating,userName,content,label
0,20220601,1,동동,타 기관 인증서 불러오기가 안되는데 대체 무슨 일이죠?? 범용공동인증서를 등록하려고...,0
1,20220626,2,Younghoon Jeon,다른건 다 모르겠고 어플 켰을 때 메인 화면이 뜨기까지 시간 좀 더 단축시켜주세요....,0
2,20220629,1,Kuee Haeng Lee,타기관인증서를 갱신하면서 인증서가져오기를 수행했는데요. QR로 가져오기는 아예 안돼...,0
3,20220709,3,brian lryan,갤럭시쓰는데 알림진동이 안 울립니다.(에티켓모드 아님) 전에 리뷰보니까 다른분들도 ...,1
4,20220625,5,박태준,뭣같은 앱아이콘을 바꾼건 정말 좋네요. 거기다 묘하게 앱 자체의 속도도 빨라진 느낌...,1
...,...,...,...,...,...
2035,20210506,1,사람이 꽃보다 아름다워,갤럭시s9+사용 고객입니다.. 케이뱅크 앱이 실행이 안되고 로고만 뜨네요.. 3일내...,0
2036,20210816,1,Call Me K,저는 당신들이 너무 싫습니다. 잘쓰고 있던거를 못쓰게 하고 돈도 못보내고 정말 당신...,0
2037,20210223,1,아주그냥,접속을 못하네 인출좀 할려고 하니 상담원은 계속 통화중... 지친다,0
2038,20210413,1,zin lee,인증번호자체가 안오네요 삼성갤럭시 S9 최신OS 시간은 오후 8시부터 계속~ 쭉이요...,0


### 불용어 제거

In [ ]:
text = ''
review = []
for each_line in df['content']:
  review.append(each_line)

In [ ]:
def clean_str(text):
  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
  text = re.sub(pattern=pattern, repl='', string=text)
  pattern = '<[^>]*>'         # HTML 태그 제거
  text = re.sub(pattern=pattern, repl='', string=text)
  pattern = '[^\w\s]'         # 특수기호제거
  text = re.sub(pattern=pattern, repl='', string=text)
  return text 

# 불용어 제거한 review 저장
review_ = []
for i in review:
  a = clean_str(i)
  review_.append(a)

In [ ]:
print("Before Stopwords : ", review[239])
print("After Stopwords : ", review_[239])

Before Stopwords :  앱이 심심하면 안열리나봐요^^ 개짜증나네요 업데이트 안하면 안열리는거면 자동 업데이트로 이어지게 창이라도 띄웠으면 합니다. 업데이트는 툭하면 하는데 할때마다 영 열리지가 않으니 쓰라는건지말라는건지
After Stopwords :  앱이 심심하면 안열리나봐요 개짜증나네요 업데이트 안하면 안열리는거면 자동 업데이트로 이어지게 창이라도 띄웠으면 합니다 업데이트는 툭하면 하는데 할때마다 영 열리지가 않으니 쓰라는건지말라는건지


### 한글 표현만 남기기

In [ ]:
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result

In [ ]:
print("Before Extraction : ",df['content'][135])
print("After Extraction : ", extract_word(df['content'][135]))
print("Before Extraction : ",df['content'][206])
print("After Extraction : ", extract_word(df['content'][206]))

Before Extraction :  알림시 진동안울림 !!!, 실행시 파란배경로고 나오기전 자주색+보라색 같은 배경이 먼저나옴 엄청 거슬림, 앱 전체가 너무 무거움 (기기 최신기종임)
After Extraction :  알림시 진동안울림      실행시 파란배경로고 나오기전 자주색 보라색 같은 배경이 먼저나옴 엄청 거슬림  앱 전체가 너무 무거움  기기 최신기종임 
Before Extraction :  업데이트 하고 나니 귀신같이 푸쉬알림 잘울림 ㅋㅋㅋㅋㅋㅋ개뿔 ㅋㅋㅋ 다시 고요해짐 ㅋㅋㅋ
After Extraction :  업데이트 하고 나니 귀신같이 푸쉬알림 잘울림       개뿔     다시 고요해짐    


### 띄어쓰기 고치기

In [ ]:
def no_space(text):
  text1 = re.sub('&nbsp;|&nbsp;|\n|\t|\r', '', text)
  text2 = re.sub('\n\n','', text1)
  return text2

In [ ]:
spacing = Spacing()

print("Before Fixing : ",df['content'][123])
print("After Fixing : ", spacing(df['content'][123]))
print("Before Fixing : ",df['content'][258])
print("After Fixing : ", spacing(df['content'][258]))

Before Fixing :  앱딱열면첫페이지가갑자기보이지않아요 괜찮았는데어느순간안되서삭제후다시깔았는데도 첫페이지는하얗게나와요. ㅠ계좌내역등을조회하려면마이페이지쪽으로가야만보여요ㅠㅠ
After Fixing :  앱 딱 열면 첫 페이지가 갑자기 보이지 않아 요 괜찮았는데 어느 순간 안 되서 삭제 후 다시 깔았는데도 첫 페이지는 하얗게 나와요. ㅠ 계좌 내역 등을 조회하려면 마이페이지 쪽으로 가야만 보여 요ㅠㅠ
Before Fixing :  케이뱅크켜면 얼마전부터 네트워크상태가 안좋다면서 다시깔아야해요 그리고는 초기로그인 딱한번만 제대로되요 데이터로도 와이파이로도 장소도바꿔봤지만 계속그러네요 업데이트때문인지 불편해요
After Fixing :  케이뱅크 켜면 얼마 전부터 네트워크 상태가 안 좋다면서 다시 깔아야 해요 그리고는 초기로그인 딱 한 번만 제대로 되요 데이터로도 와이파이로도 장소도 바꿔봤지만 계속 그러네요 업데이트 때문인지 불편해요


In [ ]:
# 전체 데이터에 적용
df['content'] = df['content'].apply(lambda x:clean_str(x))
df['content'] = df['content'].apply(lambda x:extract_word(x))
df['content'] = df['content'].apply(lambda x:no_space(x))

### 형태소 추출

In [ ]:
a = len(review_)  # 불용어 제거한 리뷰
token_review_list=[]

for i in range(0,a):
  token_review = t.morphs(review_[i])
  token_review_list.append(token_review)

df['token_review'] = token_review_list

In [ ]:
df

,date,rating,userName,content,label,token_review
0,20220601,1,동동,타 기관 인증서 불러오기가 안되는데 대체 무슨 일이죠 범용공동인증서를 등록하려고 하...,0,"[타, 기관, 인증서, 불러, 오기, 가, 안되는데, 대체, 무슨, 일이, 죠, 범..."
1,20220626,2,Younghoon Jeon,다른건 다 모르겠고 어플 켰을 때 메인 화면이 뜨기까지 시간 좀 더 단축시켜주세요 ...,0,"[다른건, 다, 모르겠고, 어플, 켰을, 때, 메인, 화면, 이, 뜨기까지, 시간,..."
2,20220629,1,Kuee Haeng Lee,타기관인증서를 갱신하면서 인증서가져오기를 수행했는데요 로 가져오기는 아예 안돼고...,0,"[타, 기관, 인증서, 를, 갱신, 하면서, 인증서, 가져오기를, 수행, 했는데요,..."
3,20220709,3,brian lryan,갤럭시쓰는데 알림진동이 안 울립니다에티켓모드 아님 전에 리뷰보니까 다른분들도 동일증...,1,"[갤럭시, 쓰는데, 알림, 진동, 이, 안, 울립니다, 에티켓, 모드, 아님, 전,..."
4,20220625,5,박태준,뭣같은 앱아이콘을 바꾼건 정말 좋네요 거기다 묘하게 앱 자체의 속도도 빨라진 느낌이...,1,"[뭣같, 은, 앱, 아이콘, 을, 바꾼건, 정말, 좋네요, 거기, 다, 묘, 하게,..."
...,...,...,...,...,...,...
2035,20210506,1,사람이 꽃보다 아름다워,갤럭시 사용 고객입니다 케이뱅크 앱이 실행이 안되고 로고만 뜨네요 일내내빠른 시...,0,"[갤럭시, s, 9, 사용, 고객, 입니다, 케이, 뱅크, 앱, 이, 실행, 이, ..."
2036,20210816,1,Call Me K,저는 당신들이 너무 싫습니다 잘쓰고 있던거를 못쓰게 하고 돈도 못보내고 정말 당신들...,0,"[저, 는, 당신, 들, 이, 너무, 싫습니다, 잘쓰고, 있던거를, 못, 쓰게, 하..."
2037,20210223,1,아주그냥,접속을 못하네 인출좀 할려고 하니 상담원은 계속 통화중 지친다,0,"[접속, 을, 못, 하네, 인, 출, 좀, 할려고, 하니, 상담, 원, 은, 계속,..."
2038,20210413,1,zin lee,인증번호자체가 안오네요 삼성갤럭시 최신 시간은 오후 시부터 계속 쭉이요 ...,0,"[인증, 번호, 자체, 가, 안오네요, 삼성, 갤럭시, S, 9, 최신, OS, 시..."


### 학습 데이터 구분

In [ ]:
train_data,test_data=train_test_split(df,test_size=.25,random_state=42)

In [ ]:
# train용 리뷰 처리
train_number_list=[]
for num in range(len(train_data)):
    train_number_list.append(num)
    

train_data['index']=train_number_list
train_data=train_data.set_index(['index'])

train_data['content'].nunique(),
train_data['rating'].nunique(),
train_data['label'].nunique()

train_data.drop_duplicates(subset=['content'],inplace=True)

In [ ]:
print("총 샘플의 개수: ",len(train_data))

총 샘플의 개수:  1530


In [ ]:
# test용 리뷰 처리
test_number_list = []

for num in range(len(test_data)):
  test_number_list.append(num)

test_data['index']=test_number_list
test_data=test_data.set_index(['index'])

test_data['content'].nunique(),
test_data['rating'].nunique()
test_data['label'].nunique()

test_data.drop_duplicates(subset=['content'],inplace=True)

In [ ]:
print("총 샘플의 개수: ",len(test_data))

총 샘플의 개수:  510


In [ ]:
# 전처리 내용 저장
test_data.to_pickle("/content/data/test_data.pkl")
train_data.to_pickle("/content/data/train_data.pkl")

In [ ]:
# 다시 읽어오기
test_data = pd.read_pickle("/content/data/test_data.pkl")
train_data = pd.read_pickle("/content/data/train_data.pkl")

In [ ]:
# train, test data 토큰화 
okt=Okt()
test_data = pd.read_pickle("/content/data/test_data.pkl")
train_data = pd.read_pickle("/content/data/train_data.pkl")
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게','요','거','로','으로',
            '것','수','할','하는','제','에서','그','데','번','해도','죠','된','건','바','구','세']
test_data['tokenized']=test_data['content'].apply(t.morphs)
test_data['tokenized']=test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
train_data['tokenized']=train_data['content'].apply(okt.morphs)
train_data['tokenized']=train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])